In [12]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import os
import mediapipe as mp

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize hand detector
detector = HandDetector(maxHands=1)

# Constants
imgSize = 500
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/data_est"
# List of letters A-Y (adjust as needed)
letters = [chr(i) for i in range(ord('A'), ord('Y') + 1)]
maxImages = 600          # Total images to capture per class
paddingFactor = 0.45     # Padding percentage

mp_hands = mp.solutions.hands

# Function to process and resize images (canvas will match input channels)
def process_and_resize(imgCrop, aspectRatio, imgSize):
    channels = 1 if len(imgCrop.shape) == 2 else imgCrop.shape[2]
    imgWhite = np.ones((imgSize, imgSize, channels), np.uint8) * 0
    try:
        if aspectRatio > 1:
            # If height > width:
            k = imgSize / imgCrop.shape[0]
            wCal = math.ceil(k * imgCrop.shape[1])
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
        else:
            # If width >= height:
            k = imgSize / imgCrop.shape[1]
            hCal = math.ceil(k * imgCrop.shape[0])
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
    except Exception as e:
        print(f"Error during image processing: {e}")
        return None
    return imgWhite

# Function to detect skin using YCrCb thresholds (returns a binary mask)
def detect_skin(frame):
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    lower_skin = np.array([0, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)
    mask = cv2.inRange(ycrcb, lower_skin, upper_skin)
    
    # Clean up noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    
    # (Optional) clean-up by drawing filled contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_mask = np.zeros_like(mask)
    cv2.drawContours(contour_mask, contours, -1, 255, thickness=cv2.FILLED)
    mask = cv2.bitwise_and(mask, contour_mask)
    
    return mask

# Main loop for each letter
for className in letters:
    print(f"Starting collection for: {className}")
    folder = os.path.join(baseFolder, className)
    os.makedirs(folder, exist_ok=True)

    counter, collecting = 0, False

    while counter < maxImages:
        success, img = cap.read()
        if not success:
            print("Camera access failed.")
            break

        # Detect hand on the full image (only once)
        hands, _ = detector.findHands(img, draw=False)
        if hands:
            # Use the first detected hand
            hand = hands[0]
            bbox = hand['bbox']       # [x, y, w, h]
            lm_list = hand['lmList']    # list of landmarks in full-image coordinates
            x, y, w, h = bbox

            # Calculate padding (based on hand size)
            xPad = int(w * paddingFactor)
            yPad = int(h * paddingFactor)

            # Compute crop boundaries (make sure they’re within image bounds)
            crop_x1 = max(0, x - xPad)
            crop_y1 = max(0, y - yPad)
            crop_x2 = min(x + w + xPad, img.shape[1])
            crop_y2 = min(y + h + yPad, img.shape[0])
            imgCrop = img[crop_y1:crop_y2, crop_x1:crop_x2]

            if imgCrop.size > 0:
                # ----- STEP 1: Draw landmarks on the cropped image using adjusted coordinates -----
                # Create a copy of the crop on which we will draw the landmarks
                imgCrop_landmarked = imgCrop.copy()
                # Adjust each landmark from full-image coordinates to crop coordinates
                for lm in lm_list:
                    adj_x = lm[0] - crop_x1
                    adj_y = lm[1] - crop_y1
                    cv2.circle(imgCrop_landmarked, (adj_x, adj_y), 4, (0, 0, 255), -1)
                # Optionally, also draw the connections between landmarks:
                for connection in mp.solutions.hands.HAND_CONNECTIONS:
                    pt1 = lm_list[connection[0]]
                    pt2 = lm_list[connection[1]]
                    pt1_adjusted = (pt1[0] - crop_x1, pt1[1] - crop_y1)
                    pt2_adjusted = (pt2[0] - crop_x1, pt2[1] - crop_y1)
                    cv2.line(imgCrop_landmarked, pt1_adjusted, pt2_adjusted, (0, 0, 255), 2)

                # ----- STEP 2: Convert the landmarked crop to a binary image -----
                # First, get a binary skin mask from the original crop (without landmarks)
                binaryMask = detect_skin(imgCrop)
                # Create a blank (black) image and fill white where skin is detected
                binary_result = np.zeros_like(imgCrop)
                binary_result[binaryMask > 0] = [255, 255, 255]
                # Now overlay the landmarks (drawn in black) onto the binary image.
                # (Using the same adjusted coordinates from above)
                for lm in lm_list:
                    adj_x = lm[0] - crop_x1
                    adj_y = lm[1] - crop_y1
                    cv2.circle(binary_result, (adj_x, adj_y), 4, (0, 0, 0), -1)
                for connection in mp.solutions.hands.HAND_CONNECTIONS:
                    pt1 = lm_list[connection[0]]
                    pt2 = lm_list[connection[1]]
                    pt1_adjusted = (pt1[0] - crop_x1, pt1[1] - crop_y1)
                    pt2_adjusted = (pt2[0] - crop_x1, pt2[1] - crop_y1)
                    cv2.line(binary_result, pt1_adjusted, pt2_adjusted, (0, 0, 0), 2)

                # ----- STEP 3: Resize for saving/visualization -----
                # Use the crop’s aspect ratio for correct resizing. (You can also use h/w from bbox.)
                aspectRatio = (crop_y2 - crop_y1) / (crop_x2 - crop_x1)
                imgWhite = process_and_resize(binary_result, aspectRatio, imgSize)
                if imgWhite is not None:
                    cv2.imshow("Processed Binary Image", imgWhite)
                    if collecting:
                        counter += 1
                        savePath = os.path.join(folder, f"{className.lower()}_{counter}.jpg")
                        cv2.imwrite(savePath, imgWhite)
                        print(f"Saved {counter}/{maxImages} images for {className}")

        # Show the original live feed
        cv2.imshow("Live Feed with Landmarks", img)
        key = cv2.waitKey(1)
        if key == ord('s'):
            collecting = True
        if key == ord('p'):
            collecting = False

    print(f"Completed collection for {className}")
    input("Press Enter for next class.")

cap.release()
cv2.destroyAllWindows()


Starting collection for: A
Saved 1/600 images for A
Saved 2/600 images for A
Saved 3/600 images for A
Saved 4/600 images for A
Saved 5/600 images for A
Saved 6/600 images for A
Saved 7/600 images for A
Saved 8/600 images for A
Saved 9/600 images for A
Saved 10/600 images for A
Saved 11/600 images for A
Saved 12/600 images for A
Saved 13/600 images for A
Saved 14/600 images for A
Saved 15/600 images for A
Saved 16/600 images for A
Saved 17/600 images for A
Saved 18/600 images for A
Saved 19/600 images for A
Saved 20/600 images for A
Saved 21/600 images for A
Saved 22/600 images for A
Saved 23/600 images for A
Saved 24/600 images for A
Saved 25/600 images for A
Saved 26/600 images for A
Saved 27/600 images for A
Saved 28/600 images for A
Saved 29/600 images for A
Saved 30/600 images for A
Saved 31/600 images for A
Saved 32/600 images for A
Saved 33/600 images for A
Saved 34/600 images for A
Saved 35/600 images for A
Saved 36/600 images for A
Saved 37/600 images for A
Saved 38/600 images 